# EX8-STREAM: Spark Structured Streaming basics

Your assignment: complete the `TODO`'s and include also the **output of each cell**.

#### You may need to read the [Structured Streaming API Documentation](https://spark.apache.org/docs/latest/api/python/reference/pyspark.ss/index.html) to complete this lab.

### Step 2: Start Spark Session

In [7]:
from pyspark.sql import SparkSession

try:
    spark.stop()
except NameError:
    print("SparkContext not defined")

# local mode
spark = SparkSession.builder \
            .appName("Spark SQL basic example") \
            .master("local[*]") \
	    	.config("spark.some.config.option", "some-value") \
	    	.getOrCreate()

# cluster mode
#spark = SparkSession.builder \
#            .appName("Spark SQL basic example") \
#            .master("spark://spark:7077") \
#	    	.config("spark.some.config.option", "some-value") \
#	    	.getOrCreate()

In [8]:
lines = (spark
.readStream.format("socket")
.option("host", "localhost")
.option("port", 9999)
.load())

lines

25/03/21 22:18:39 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


DataFrame[value: string]

In [9]:
from pyspark.sql.functions import *
#words = lines.select(explode(split(lines.value, "\\s")).alias("word"))
words = lines.select(split(lines.value, "\\s").alias("key_value"))
words = words.select(words.key_value.getItem(0).alias("key"), words.key_value.getItem(1).alias("value"))
counts = words.groupBy("key").count()
counts

DataFrame[key: string, count: bigint]

In [10]:
writer = counts.writeStream.format("console").outputMode("complete")
writer

In [11]:
checkpointDir = "/tmp/structured_streaming"
writer2 = (writer
.trigger(processingTime="5 second")
.option("checkpointLocation", checkpointDir)
          )
writer2

In [ ]:
!rm -r /tmp/structured_streaming/
streamingQuery = writer2.start()
streamingQuery.awaitTermination()

25/03/21 22:18:40 ERROR MicroBatchExecution: Query [id = be0c67dd-8c73-4f9b-961b-5f2d32b3697d, runId = 9135dd5f-800e-405e-8a04-daee1c90a83f] terminated with error
java.io.FileNotFoundException: File file:/tmp/structured_streaming/offsets does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:779)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1100)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:769)
	at org.apache.hadoop.fs.DelegateToFileSystem.getFileStatus(DelegateToFileSystem.java:128)
	at org.apache.hadoop.fs.DelegateToFileSystem.createInternal(DelegateToFileSystem.java:93)
	at org.apache.hadoop.fs.ChecksumFs$ChecksumFSOutputSummer.<init>(ChecksumFs.java:353)
	at org.apache.hadoop.fs.ChecksumFs.createInternal(ChecksumFs.java:400)
	at org.apache.hadoop.fs.AbstractFileSystem.create(AbstractFileSystem.java:626)
	at org.apache.hadoop.fs.FileContext$3.nex

-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+
|key|count|
+---+-----+
+---+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
|  key|count|
+-----+-----+
|Hello|    5|
+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
|  key|count|
+-----+-----+
|Hello|   10|
+-----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-----+-----+
|  key|count|
+-----+-----+
|Hello|   15|
+-----+-----+



25/03/21 22:19:00 WARN TextSocketMicroBatchStream: Stream closed by localhost:9999
                                                                                

-------------------------------------------
Batch: 4
-------------------------------------------
+-----+-----+
|  key|count|
+-----+-----+
|Hello|   20|
+-----+-----+

